In [1]:
import pickle
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from features.pkl import PickleFeatureWriter
from datasets.pickle_dataset import MultiPickleDataset
import os
from torch.utils.data import DataLoader
from pathlib import Path
from torch.optim import Adam
import torch
from frame_sampling import RandomSampler
from scripts.train_mtrn import train_test_loader
from systems_beta import EpicActionRecogintionShapleyClassifier
from livelossplot import PlotLosses
import numpy as np
from torchvideo.samplers import frame_idx_to_list

In [31]:
train = Path('/mnt/storage/home/jb18789/scratch/datasets/epic-100/features/67217_train_features.pkl')
test = Path('/mnt/storage/home/jb18789/scratch/datasets/epic-100/features/9668_val_features.pkl')

p01 = Path('/mnt/storage/scratch/jb18789/datasets/epic-100/video_id_features/5509_p01_features_new.pkl')
# with open(train, 'rb') as f:
#     df = pickle.load(f)

# p.parent / f'{id}.pkl'\

In [32]:
dataset = MultiPickleDataset(p01)

In [57]:
trainloader, testloader = train_test_loader(dataset, batch_size=512, val_split=0.3)
x = np.array([x['num_frames'] for x in dataset.pkl_dict['labels']])

x.mean() / 4

46.93669449990924

In [ ]:
i = iter(trainloader)
xd = i.next()

xd[0][0].shape

In [58]:
class V_MTRN(nn.Module):
    
    def __init__(self, frame_count: int):
        super().__init__()
        self.frame_count = frame_count
        self.fc1 = nn.Linear(256 * frame_count, 1024)
#         self.fc1_2 = nn.Linear(4096, 1024)
#         self.d1 = nn.Dropout(p=0.7)
        self.d2 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3_verb = nn.Linear(512, 97)
    
    def forward(self, x):
        x = x.view(-1, 256 * self.frame_count)
        x = F.relu(self.fc1(x))
#         x = self.d1(x)
#         x = F.relu(self.fc1_2(x))
        x = self.d2(x)
        x = F.relu(self.fc2(x))
        x = self.d2(x)
#         x = self.dropout(x)
        x = self.fc3_verb(x)
        
        return x
    
class V_P01(nn.Module):
    def __init__(self, frame_count:int):
        super().__init__()
        self.frame_count = frame_count
        self.fc1 = nn.Linear(256*frame_count, 512)
        self.fc2 = nn.Linear(512,97)
    def forward(self, x):
        x = x.view(-1, 256*self.frame_count)
        return self.fc2(F.relu(self.fc1(x)))
    
class N_P01(nn.Module):
    def __init__(self, frame_count:int):
        super().__init__()
        self.frame_count = frame_count
        self.fc1 = nn.Linear(256*frame_count, 512)
        self.fc2 = nn.Linear(512,300)
    def forward(self, x):
        x = x.view(-1, 256*self.frame_count)
        return self.fc2(F.relu(self.fc1(x)))
    
class P01(nn.Module):
    def __init__(self, frame_count:int):
        super().__init__()
        self.frame_count = frame_count
        self.fc1 = nn.Linear(256*frame_count, 512)
        self.fc2 = nn.Linear(512,397)
    def forward(self, x):
        x = x.view(-1, 256*self.frame_count)
        return self.fc2(F.relu(self.fc1(x)))
    
class N_MTRN(nn.Module):
    
    def __init__(self, frame_count: int):
        super().__init__()
        self.frame_count = frame_count
        self.fc1 = nn.Linear(256 * frame_count, 1024)
#         self.fc1_2 = nn.Linear(4096, 1024)
#         self.d1 = nn.Dropout(p=0.7)
        self.d2 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3_noun = nn.Linear(512, 300)
    
    def forward(self, x):
        x = x.view(-1, 256 * self.frame_count)
        x = F.relu(self.fc1(x))
#         x = self.d1(x)
#         x = F.relu(self.fc1_2(x))
        x = self.d2(x)
        x = F.relu(self.fc2(x))
        x = self.d2(x)
#         x = self.dropout(x)
        x = self.fc3_noun(x)
        
        return x

In [59]:
def no_collate(args):
    return args

In [60]:
v_models = [V_MTRN(frame_count=i) for i in range(1,9)]
n_models = [N_MTRN(frame_count=i) for i in range(1,9)]
v_p01 = [V_P01(frame_count=i) for i in range(1,9)]
n_p01 = [N_P01(frame_count=i) for i in range(1,9)]
m_p01 = [P01(frame_count=i) for i in range(1,9)]
v_optims = [Adam(m.parameters(), lr=5e-5) for m in v_p01]
n_optims = [Adam(m.parameters(), lr=5e-5) for m in n_p01]
test_samplers = [RandomSampler(frame_count=m.frame_count, snippet_length=1,test=True) for m in v_models]
train_samplers = [RandomSampler(frame_count=m.frame_count, snippet_length=1,test=False) for m in v_models]

In [ ]:
trainloader = DataLoader(MultiPickleDataset(train), batch_size=512, collate_fn=no_collate, shuffle=True)
testloader = DataLoader(MultiPickleDataset(test), batch_size=512, collate_fn=no_collate, shuffle=False)

In [54]:
# trainloader, testloader = train_test_loader(dataset, 1, 0.3)
# print(testloader.sampler)
# print(len(trainloader.sampler.num_samples))
# print(len(testloader.sampler.indicies))

In [61]:
classifier = EpicActionRecogintionShapleyClassifier(
    v_p01[3],
    torch.device('cpu'),
    v_optims[3],
    train_samplers[3],
    test_samplers[3],
    trainloader,
    testloader,
    'verb'
)


Classifier, model: <class '__main__.V_P01'>, type: verb, frames: 4


In [ ]:
# from sklearn.utils import class_weight

test_classes = torch.tensor([label['verb_class'] for label in trainloader.dataset.pkl_dict['labels']])

t1 = torch.tensor(test_classes)

# weights = class_weight.compute_class_weight('balanced',classes=t1.unique().numpy(), y=t1.numpy())
weights = len(t1)/(97*t1.bincount(minlength=97))

# t1.bincount(minlength=97)
# torch.tensor(weights, dtype=)
# np.arange(0,97,1)
# weights
classifier.train_weights
# models[3]

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
lslsls = Counter(all_classes)

In [ ]:
xd = iter(trainloader)
data, labels = classifier._sample_frames(xd.next())

In [ ]:

labels
# pxx = torch.cat([x['verb_class'] for x in labels])

out = classifier.model(data)
# weights[pxx]
import torch.nn.functional as F

F.cross_entropy(out, labels['verb_class'],weight=weights)
# out[:,1] * weights[1]

In [ ]:
# alie = torch.tensor(list(dict(sorted(lslsls.items())).items()))
# fig, ax = plt.subplots()

# ax.plot(alie[:,1])
# trainloader.dataset.pkl_dict['labels']
# models[3]

# len(classifier.trainloader.dataset)

In [ ]:
class h_mask(object):
    def __init__(self, p: float=0.5):
        self.p = p
        
    def __call__(self, sample):
        x = np.random.choice([0,1], p=[1-self.p, self.p])
        if x:
            

In [ ]:
it = iter(trainloader)
x = it.next()

In [ ]:
def _sample_frames(data):
    features = []
    labels = {}
    for feature, label in data:
        video_length = feature.shape[0]
        if video_length < frame_samplers[3].frame_count:
            raise ValueError(f"Video too short to sample {frame_samplers[3].frame_count} from")
        idxs = np.array(frame_idx_to_list(frame_samplers[3].sample(video_length)))
        features.append(feature[idxs])
        for k in label.keys():
            if k in labels:
                labels[k].append(label[k])
            else:
                labels[k] = [label[k]]

    for k in labels.keys():
        try:
            labels[k] = torch.tensor(labels[k])
        except ValueError:
            pass

    return torch.tensor(features, dtype=torch.float), labels

In [ ]:
alie = _sample_frames(x)

In [ ]:
# data, labels = _sample_frames(x)
alie = classifier.test_step()

In [ ]:
torch.save(classifier.model.state_dict(),'mtrn_verb_frames=4_lr=1e-4_hl=1024_dc=2.pt')

In [ ]:
# np.array(list(alie.values())).mean(1)

In [ ]:
classifier.model

In [ ]:
1 % 1000 == 1